In [2]:
! pip install -Uq pandas fastparquet

In [3]:
import pandas as pd

In [4]:
DATA_PATH = '../data'

In [5]:
! ls {DATA_PATH}

data_cleaned.csv  data_raw.parquet


In [7]:
df = pd.read_parquet(f'{DATA_PATH}/data.parquet', engine='fastparquet')
df

,timestamp,country_code,last_order_ts,first_order_ts,total_orders,voucher_amount
0,2020-05-20 15:43:38.364972+00:00,China,2020-04-19 00:00:00+00:00,2020-04-18 00:00:00+00:00,0.0,5720.0
1,2020-05-20 15:43:47.914346+00:00,China,2020-04-19 00:00:00+00:00,2020-04-13 00:00:00+00:00,1.0,8800.0
2,2020-05-20 15:36:56.866239+00:00,China,2020-04-19 00:00:00+00:00,2020-04-15 00:00:00+00:00,0.0,1760.0
3,2020-05-20 15:00:33.694108+00:00,China,2020-04-19 00:00:00+00:00,2020-04-13 00:00:00+00:00,,1760.0
4,2020-05-20 15:43:18.521402+00:00,China,2020-04-19 00:00:00+00:00,2020-04-14 00:00:00+00:00,3.0,8800.0
...,...,...,...,...,...,...
511422,2020-04-03 18:31:36.635072+00:00,Latvia,2020-01-04 00:00:00+00:00,2019-10-30 00:00:00+00:00,27.0,4400.0
511423,2020-04-03 18:33:32.365196+00:00,Latvia,2020-01-04 00:00:00+00:00,2019-08-29 00:00:00+00:00,10.0,4400.0
511424,2020-04-03 19:23:07.711424+00:00,Peru,2020-01-04 00:00:00+00:00,2019-10-14 00:00:00+00:00,47.0,4400.0
511425,2020-04-03 19:19:06.162203+00:00,Australia,2020-01-04 00:00:00+00:00,2019-09-24 00:00:00+00:00,2.0,11000.0


---
Now we want to cleanup timestamp columns to `datetime`:

In [8]:
df[df['timestamp'].isna()].count()

timestamp         0
country_code      0
last_order_ts     0
first_order_ts    0
total_orders      0
voucher_amount    0
dtype: int64

In [9]:
df[df['first_order_ts'].isna()].count()

timestamp         0
country_code      0
last_order_ts     0
first_order_ts    0
total_orders      0
voucher_amount    0
dtype: int64

In [10]:
df[df['last_order_ts'].isna()].count()

timestamp         0
country_code      0
last_order_ts     0
first_order_ts    0
total_orders      0
voucher_amount    0
dtype: int64

OK so there's no NaN timestamps, we can safely convert it to `datetime`

In [11]:
for col in ['timestamp', 'first_order_ts', 'last_order_ts']:
    df[col] = pd.to_datetime(df[col])

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511427 entries, 0 to 511426
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype              
---  ------          --------------   -----              
 0   timestamp       511427 non-null  datetime64[ns, UTC]
 1   country_code    511427 non-null  object             
 2   last_order_ts   511427 non-null  datetime64[ns, UTC]
 3   first_order_ts  511427 non-null  datetime64[ns, UTC]
 4   total_orders    511427 non-null  object             
 5   voucher_amount  481860 non-null  float64            
dtypes: datetime64[ns, UTC](3), float64(1), object(2)
memory usage: 23.4+ MB


---
Now we want to cleanup column `total_orders`: type should be `int`

In [13]:
df['total_orders'].dtype

dtype('O')

So the column `total_orders` is of type Python object. We want to convert it all to `int` if possible without information loss, otherwise to `float`.

In [14]:
## The following code fails with:
## 'ValueError: could not convert string to float: '
## which means that some objects are None or empty strings or whatever.

# df['total_orders'].astype('float')

In [15]:
# Total:
df['total_orders'].count()

511427

In [16]:
# NaN:
df[ df['total_orders'].isna() ]['timestamp'].count()

0

In [17]:
# None:
df['total_orders'].apply(lambda x: x is None).sum()

0

In [18]:
# Empty strings:
df['total_orders'].apply(lambda x: x == '').sum()

40914

OK so the wrong objects are empty strings. We'll set `0.0` instead.

In [19]:
# Let's see if we can convert all strings to floats and see if they can be integers:
df['total_orders'].apply(lambda x: float(x or '0.0').is_integer()).all()

True

The check above shows that now all `total_orders` can be converted to `int`. Let's do it:

In [20]:
# Convert this column to `int`:
df['total_orders'] = df['total_orders'].apply(lambda x: int(float(x or '0.0')))
df[:3]

,timestamp,country_code,last_order_ts,first_order_ts,total_orders,voucher_amount
0,2020-05-20 15:43:38.364972+00:00,China,2020-04-19 00:00:00+00:00,2020-04-18 00:00:00+00:00,0,5720.0
1,2020-05-20 15:43:47.914346+00:00,China,2020-04-19 00:00:00+00:00,2020-04-13 00:00:00+00:00,1,8800.0
2,2020-05-20 15:36:56.866239+00:00,China,2020-04-19 00:00:00+00:00,2020-04-15 00:00:00+00:00,0,1760.0


---
Now let's apply the same cleanup to the column `voucher_amount`

In [21]:
# False (casted) objects:
df['voucher_amount'].apply(lambda x: not x).sum()

0

In [22]:
# NaN:
df[ df['voucher_amount'].isna() ]['timestamp'].count()

29567

In [23]:
# Let's replace NaN values with zeros:
df['voucher_amount'] = df['voucher_amount'].fillna(0)
df[ df['voucher_amount'].isna() ]['timestamp'].count()

0

In [24]:
# Now, let's check if we can convert all floats to integer without data loss:
df['voucher_amount'].apply(lambda x: float(x).is_integer()).value_counts()

True    511427
Name: voucher_amount, dtype: int64

In [25]:
# Now, let's fix the column `voucher_amount`:
df['voucher_amount'] = df['voucher_amount'].apply(lambda x: int(float(x)))
df[:3]

,timestamp,country_code,last_order_ts,first_order_ts,total_orders,voucher_amount
0,2020-05-20 15:43:38.364972+00:00,China,2020-04-19 00:00:00+00:00,2020-04-18 00:00:00+00:00,0,5720
1,2020-05-20 15:43:47.914346+00:00,China,2020-04-19 00:00:00+00:00,2020-04-13 00:00:00+00:00,1,8800
2,2020-05-20 15:36:56.866239+00:00,China,2020-04-19 00:00:00+00:00,2020-04-15 00:00:00+00:00,0,1760


---
Now, let's explore the column `country_code`

In [26]:
df['country_code'].value_counts()

China        169722
Australia    134004
Peru         106547
Latvia       101154
Name: country_code, dtype: int64

In [27]:
# Check NaN:
df['country_code'].isna().sum()

0

OK so this column is clean: all rows have a valid value.

---
Let's save the cleaned data to a csv file.

In [32]:
df.to_csv(f'{DATA_PATH}/data_clean.csv')

In [33]:
! ls {DATA_PATH}

data_clean.csv	data.parquet
